# Exercise 3: Mathematical Problem Solving with LLMs

**This is a marked exercise (graded)**

Apply LLMs to solve mathematical reasoning tasks. Test different pre-trained models with various prompting strategies and optionally fine-tune with LoRA to improve performance.

**Learning Objectives:**
- Evaluate LLMs on mathematical reasoning
- Design effective prompts for numerical tasks
- Implement and compare different prompting strategies
- Optionally: Fine-tune models using LoRA
- Measure performance using accuracy metric with tolerance

**Deliverables:**
- Completed notebook with your approach
- `submission.csv` with predictions on test set (100 problems)
- Score: Accuracy with 2 decimal precision tolerance (threshold: 70%)

## Part 1: Setup and Load Data

In [ ]:
!pip install transformers torch peft datasets pandas scikit-learn matplotlib requests -q

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import requests
import re

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## Part 2: Download Dataset

Download the math problem dataset (1000 problems: 900 train, 100 test).

In [ ]:
# URLs for the dataset files
base_url = 'https://www.raphaelcousin.com/modules/data-science-practice/module8/exercise/'

train_url = base_url + 'train.csv'
test_url = base_url + 'test.csv'

def download_file(url, filename):
    """Download a file from URL."""
    response = requests.get(url)
    response.raise_for_status()
    with open(filename, 'wb') as f:
        f.write(response.content)
    print(f"Downloaded {filename}")

# Download files
download_file(train_url, 'train.csv')
download_file(test_url, 'test.csv')

In [ ]:
# Load the datasets
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

print(f"Train set size: {len(train_data)}")
print(f"Test set size: {len(test_data)}")

# Display category distribution
print("\nTraining set category distribution:")
print(train_data['category'].value_counts().sort_index())

print("\nSample training problems:")
print(train_data.head(10))

## Part 3: Baseline - Dummy Model

Create a baseline to understand what poor performance looks like.

In [ ]:
def check_accuracy(predictions, ground_truth, tolerance=0.01):
    """
    Calculate accuracy with tolerance for floating point comparisons.

    Two values are considered equal if their difference is <= tolerance
    OR if they round to the same value at 2 decimal places.
    """
    correct = 0
    for pred, truth in zip(predictions, ground_truth):
        # Check if both round to same 2 decimal places
        if round(pred, 2) == round(truth, 2):
            correct += 1
        # Or if absolute difference is very small
        elif abs(pred - truth) <= tolerance:
            correct += 1

    return correct / len(predictions)

# Dummy baseline: always predict the mean
mean_solution = train_data['solution'].mean()
print(f"Dummy model (always predicts mean): {mean_solution:.2f}")
print("This demonstrates very poor performance. Your model should do much better!")

## Part 4: Utility Functions

Helper functions to extract numerical answers from model outputs.

In [ ]:
def extract_number(text):
    """
    Extract the first number from text. Return None if no number found.

    Handles various formats:
    - "The answer is 42"
    - "42"
    - "= 42"
    - "Result: 42.5"
    - Negative numbers: "-15"
    """
    # Try different patterns in order of specificity
    patterns = [
         r'-?\d+(?:\.\d+)?',
        r'^(-?\d+\.?\d*)\s*',
        r'(?:answer|result|equals?|=)\s*:?\s*(-?\d+\.?\d*)',  # "answer is 42" or "= 42"
        r'(-?\d+\.?\d*)\s*$',  # Number at the end
        r'(-?\d+\.?\d*)'  # Any number

    ]

    for pattern in patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            try:
                return float(match.group(1))
            except (ValueError, IndexError):
                continue

    return None

# Test extraction
test_strings = [
    "The answer is 42",
    "42",
    "15 + 27 = 42",
    "Calculating... the result is 42.5!",
    "No number here",
    "The value is -15"
]

print("Number extraction tests:")
for s in test_strings:
    result = extract_number(s)
    print(f"  '{s}' -> {result}")

## Part 5: Load Pre-trained Model

Load a small, efficient model for math problem solving.

In [ ]:
# TODO: Load a pre-trained model
# Suggested models:
# - "gpt2" (small, fast)
# - "microsoft/phi-2" (better reasoning, needs more memory)
# - "TinyLlama/TinyLlama-1.1B-Chat-v1.0" (good balance)

model_name = "Qwen/Qwen2.5-Math-1.5B" #"Qwen/Qwen2.5-Math-1.5B-Instruct" #"TinyLlama/TinyLlama-1.1B-Chat-v1.0" #"microsoft/phi-2" #"gpt2"  # Start with GPT-2

print(f"Loading model: {model_name}...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model = model.to(device)

# Set padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"Model loaded successfully!")
print(f"Model size: {sum(p.numel() for p in model.parameters()) / 1e6:.1f}M parameters")

## Part 6: Prompting Strategies

Test different prompt templates to improve model performance.

In [ ]:
def generate_answer(problem, prompt_template="simple", max_new_tokens=50, temperature=0.1):
    """
    Generate answer using different prompt templates.

    Templates:
    - simple: Just the problem
    - instruction: Add instruction to solve
    - cot: Chain-of-thought prompting
    - few_shot: Include examples from training data
    """
    if prompt_template == "simple":
        prompt = f"{problem}\nAnswer:"

    elif prompt_template == "instruction":
        prompt = f"Solve this math problem and provide only the numerical answer.\n\nProblem: {problem}\nAnswer:"

    elif prompt_template == "cot":
        prompt = f"Solve this math problem step by step, then provide the final numerical answer.\n\nProblem: {problem}\nSolution:\n"

    elif prompt_template == "few_shot":
        # Include 3-5 examples from training data
        examples = []
        for i in range(min(5, len(train_data))):
            examples.append(f"Problem: {train_data['problem'].iloc[i]}\nAnswer: {train_data['solution'].iloc[i]}")

        examples_text = "\n\n".join(examples)

        # Prompt
        prompt = (
      "You are a helpful math assistant. "
      "Read each problem carefully and answer only the last problem with the numeric result. "
    "Do not show your reasoning or steps. "
    "Your final output must contain only the number (no words or punctuation).\n\n"
    "Here are some examples of math problems and their answers:\n\n"
    f"{examples_text}\n\n"
    "---\n\n"
    "Now, solve the following problem:\n"
    f"Problem: {problem}\n"
    "Answer:"
)


    else:
        prompt = problem

    # Generate
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=True if temperature > 0 else False,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Remove the prompt from response
    response = response[len(prompt):].strip()

    return response

# Test different prompts on a sample problem
test_problem = train_data['problem'].iloc[0]
test_solution = train_data['solution'].iloc[0]

print(f"Testing problem: {test_problem}")
print(f"Correct answer: {test_solution}\n")
print("="*70)

for template in ["simple", "instruction", "cot", "few_shot"]:
    response = generate_answer(test_problem, template)
    extracted = extract_number(response)

    correct = "✓" if extracted is not None and round(extracted, 2) == round(test_solution, 2) else "✗"

    print(f"{correct} {template}:")
    print(f"  Response: {response[:100]}{'...' if len(response) > 100 else ''}")
    print(f"  Extracted: {extracted}\n")

## Part 7: Evaluate on Validation Set

Test your best prompting strategy on a subset of training data.

In [ ]:
# TODO: Choose your best prompt template
best_template = "few_shot"  # Change based on your experiments

# Evaluate on a small validation set (last 50 training examples)
val_data = train_data.tail(50)

predictions = []
ground_truth = val_data['solution'].tolist()

print(f"Evaluating on {len(val_data)} validation problems...\n")

for idx, row in val_data.iterrows():
    problem = row['problem']
    solution = row['solution']

    response = generate_answer(problem, prompt_template=best_template)
    prediction = extract_number(response)

    print(response)
    print("-------")
    print(f"Correct asnwer was : {solution}")
    print("-----")

    # If no number extracted, use 0 (will be wrong)
    if prediction is None:
        prediction = 0.0

    predictions.append(prediction)

    if (len(predictions) % 10) == 0:
        print(f"Processed {len(predictions)}/{len(val_data)} problems...")

# Calculate accuracy
accuracy = check_accuracy(predictions, ground_truth)
print(f"\nValidation Accuracy: {accuracy:.2%}")
print(f"Need to achieve: 70% on test set")

## Part 8: Generate Test Predictions

Generate predictions for the test set and create submission file.

In [ ]:
# TODO: Generate predictions on test set
print(f"Generating predictions on {len(test_data)} test problems...\n")

test_predictions = []

for idx, row in test_data.iterrows():
    problem = row['problem']

    response = generate_answer(problem, prompt_template=best_template)
    prediction = extract_number(response)

    # If no number extracted, use 0
    if prediction is None:
        prediction = 0.0
        print(f"⚠️  Warning: No number extracted for problem {idx}: {problem[:50]}...")

    test_predictions.append(prediction)

    if (idx + 1) % 10 == 0:
        print(f"Processed {idx + 1}/{len(test_data)} problems...")

print("\nAll test predictions generated!")

## Part 9: Create Submission File

Save predictions in the required format for evaluation.

In [ ]:
# Create submission DataFrame
submission = pd.DataFrame({
    'id': test_data['id'],
    'solution': test_predictions
})

# Save to CSV
submission.to_csv('submission.csv', index=False)

print("Submission file created: submission.csv")
print("\nSubmission preview:")
print(submission.head(10))

# Verify all predictions are numerical
non_numeric = submission['solution'].isna().sum()
if non_numeric > 0:
    print(f"\n⚠️  WARNING: {non_numeric} predictions are not numerical!")
    print("These will result in incorrect answers. Please fix them.")
else:
    print("\n✓ All predictions are numerical")

# Show statistics
print("\nPrediction statistics:")
print(submission['solution'].describe())

## Part 10 (Optional): Fine-Tuning with LoRA

If prompting doesn't achieve 70% accuracy, consider fine-tuning with LoRA.

In [ ]:
# TODO: Implement LoRA fine-tuning (OPTIONAL)
from peft import LoraConfig, get_peft_model, TaskType
from torch.utils.data import Dataset, DataLoader

# This is a template - implement if needed
print("LoRA fine-tuning is optional.")
print("Use this if prompting strategies don't achieve 70% accuracy.")
print("\nConsider:")
print("- Prepare training dataset in correct format")
print("- Configure LoRA parameters (r=8, alpha=32)")
print("- Train for a few epochs")
print("- Evaluate and compare with prompting approaches")

## Questions

Answer the following questions:

1. **Which prompting strategy worked best and why?**
   - La stratégie de "few_shot" a donné les meilleurs résultats.Le few-shot prompting fonctionne mieux car il montre au modèle plusieurs exemples concrets de problèmes et de réponses similaires. Cela aide le modèle à comprendre le format attendu et le type de raisonnement à appliquer.

2. **What types of math problems were most challenging for the model?**
   - Les problèmes difficiles sont ceux qui demandent plusieurs étapes (comme l'algèbre ou le calcul). Les LLM font aussi des erreurs de calcul simples, surtout avec des décimales. Les problèmes non standards (catégorie other) sont compliqués pour le modèle.

3. **How did you handle number extraction from model outputs?**
   - YOUR ANSWER HERE

4. **What are the limitations of using LLMs for mathematical reasoning?**
   - Les LLM font des erreurs de calcul. Ils peuvent "halluciner" et générer du texte en trop après la réponse. Ils imitent des exemples mais ne comprennent pas vraiment les maths. Il est parfois difficile d'extraire la réponse numérique du texte.

5. **If you used LoRA fine-tuning, what were the trade-offs compared to prompting?**
   - YOUR ANSWER HERE (or N/A if you didn't use LoRA)